In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt

from tensorflow.keras.losses import BinaryCrossentropy


# a is the actual values while b is the predictions
def bce(a, b):
    return BinaryCrossentropy(from_logits = False)(a,b).numpy()

dtypes = ['train', 'validate', 'test']

2024-05-21 21:53:27.824452: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/envs/QPL/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [502]:
dtypes = ['train', 'validate', 'test']

metric_functions = {'r': lambda x, y: np.corrcoef(x, y)[0,1],
                    'kl': lambda x, y: bce(x, y) - bce(x, x),
                    'l1': lambda x, y: np.mean(np.abs(x - y))
                   }
metric_names = {'r': 'Pearson Correlation Coefficient',
               'kl': 'Kullback-Leibler Divergence',
               'l1': 'Mean absolute error (%)'}

metric_labels = {'l1': 'Mean absolute error (\%)'}

model_predictions = {'cnn': 'processed_cnn_predictions.npz', 'custom-nn': 'processed_custom_neural_net_predictions.npz'}
model_mirrored_predictions = {'cnn': 'processed_cnn_mirrored_predictions.npz', 'custom-nn': 'processed_custom_neural_net_mirrored_predictions.npz'}

model_colors = {'cnn': (1.0, 0.4980392156862745, 0.054901960784313725), 'ground': 'tab:green', 'custom-nn': (0.12156862745098039, 0.4666666666666667, 0.7058823529411765)}

model_sizes = {'cnn': 100, 'custom-nn': 100}
model_shapes = {'cnn': 'P', 'custom-nn': 'o'}

model_alphas = {'cnn': .55, 'custom-nn': .75}
model_hist_alphas = {'cnn': .55, 'custom-nn': 1}

# circuit_error_yticks = {'random': np.linspace(-8,8,9), 'mirror-circuits': np.linspace(-8,8, 9)}
# circuit_error_xticks = {'random': np.linspace(90, 100, 6), 'mirror-circuits': np.linspace(90, 100, 6)}


model_label_names = {'cnn': 'CNN', 'custom-nn': 'qpa-NN'}

# Prediction errors

In [503]:
def error_scatter_hist(ground_truth, predictions, model_types, save = False, title = True, circuit_type = 'random'):
    fig = plt.figure(figsize=(10, 6), dpi = 300)

    gs = fig.add_gridspec(2, 2, width_ratios = (4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.05, hspace=0.05)

    ax = fig.add_subplot(gs[1, 0])
    ax_histy = fig.add_subplot(gs[0, 0], sharex=ax)
    ax_histx = fig.add_subplot(gs[1,1], sharey=ax)

    for model_type in model_types[::-1]:
        abs_error = np.abs(predictions[model_type])
        mean_abs_error = np.mean(abs_error)
        label_text = model_label_names[model_type] + ', MAE = {:.3g}%'.format(mean_abs_error)
        ax.scatter(x = ground_truth, y = predictions[model_type], color = model_colors[model_type],
                   alpha=model_alphas[model_type], marker = model_shapes[model_type], label = label_text)
        
    ### Include logic to get the window size ###
    
    x_min, x_max = min(exp_xlims[exp_num]), max(exp_xlims[exp_num])
    y_min, y_max = min(exp_ylims[exp_num]), max(exp_ylims[exp_num])

    ax.plot([x_min,x_max], [0,0], color = 'black')
    ax.fill_between([x_min, x_max], [x_max - x_min,0], [x_max - x_min,x_max - x_min], color = 'grey', alpha = .1)
    # ax.fill_between([x_min, x_max], [0,x_min - x_max], [x_min - x_max,x_min - x_max], color = 'grey', alpha = .1)

    ### Add logic to get consistent bins ###
    g_bins = np.linspace(x_min, x_max, 40)
    
    ax_histy.hist(ground_truth, bins = g_bins, color = model_colors['ground'], orientation='vertical', label = 'Fidelity (%)')
    ax_histy.legend(fontsize = 14)
        
    p_bins = np.linspace(y_min, y_max, 40)
    for model_type in model_types:
        ax_histx.hist(predictions[model_type], bins = p_bins, color = model_colors[model_type], 
                      orientation = 'horizontal', alpha = model_hist_alphas[model_type])
                
    ax_histy.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis = 'y', labelsize = 13)
    
    ax_histx.tick_params(axis="y", labelleft=False)
    ax_histx.tick_params(axis = 'x', labelsize = 13)
    ax_histx.xaxis.tick_top()

    ax.set_xlim(left = x_min, right = x_max)
    ax.set_ylim(bottom = y_min, top = y_max)

    ax.yaxis.set_ticks(exp_yticks[exp_num])
    ax.xaxis.set_ticks(exp_xticks[exp_num])

    
    ax.xaxis.set_tick_params(labelsize = 16)
    ax.yaxis.set_tick_params(labelsize = 16)
    
    ax.legend(fontsize = 18)
    ax.set_xlabel('Fidelity (%)', fontsize = 20)
    ax.set_ylabel('Prediction error (%)', fontsize = 20)
    if title is True:
        fig.suptitle('{} Error Model'.format(noise_name), fontsize = 22)
    if save is False:
        plt.show()
    else:
        plt.savefig(plot_path + 'prediction-plots/simulation-{}-{}-prediction-errors-histogram-plot.png'.format(exp_num, circuit_type), dpi = 300, bbox_inches = 'tight')
    # if 'stab' in model_types:
    #     plt.savefig(plot_path+'/non-markovian-combined-scatter-histogram-{}-noise-stab-no-stab.png'.format(noise_save_name), dpi = 300, bbox_inches = 'tight')    
    # else:
    #     plt.savefig(plot_path+'/non-markovian-combined-scatter-histogram-{}-noise.png'.format(noise_save_name), dpi = 300, bbox_inches = 'tight')
    plt.close()

# Random circuits prediction errors

In [528]:
exp_num = 4
exp_path = f'./experiment_{exp_num}/'
pred_path = exp_path + '/predictions/'
inf_path = exp_path + '/processed_inputs_and_outputs/'
plot_path = '../../../paper-plots/simulations/'

In [529]:
exp_ylims = {0: [-4.25,8.5], 1: [-4.25, 8.5], 2: [-4.25, 8.5], 3: [-4.25, 8.5], 4: [-4.25, 8.5]}
exp_yticks = {0: [-4, -2, 0, 2, 4, 6, 8], 1: [-4, -2, 0, 2, 4, 6, 8], 2: [-4, -2, 0, 2, 4, 6, 8],
              3: [-4, -2, 0, 2, 4, 6, 8], 4: [-4, -2, 0, 2, 4, 6, 8]}

exp_xlims = {0: [90, 100], 1: [92, 100], 2: [87.5, 100], 3: [92, 100], 4: [90, 100]}
exp_xticks = {0: np.linspace(90, 100, 6), 1: np.linspace(92, 100, 5), 2: np.linspace(88, 100, 4), 
              3: np.linspace(92, 100, 5), 4: np.linspace(90, 100, 6)}

In [530]:
dtypes = ['train', 'validate', 'test']

model_types = ['custom-nn', 'cnn']
circuits = 'test'

In [531]:
infidelities = np.load(inf_path + 'processed_infidelities.npz')
spss = {dt: 100*(1 - infidelities[dt] / 10000) for dt in dtypes}

m_predictions = {m: None for m in model_types}

for model in model_types:
    predictions = np.load(pred_path + model_predictions[model])

    if model in ['custom-nn']:
        m_predictions[model] = 100*(1 - predictions[circuits] / 10000) - spss[circuits]
    else:
        m_predictions[model] = 100*(1 - predictions[circuits] / 10000).reshape((-1,)) - spss[circuits]

In [532]:
error_scatter_hist(ground_truth=spss[circuits], predictions=m_predictions,
             model_types = model_types, save = True, title = False)

In [533]:
print(f'Done with {exp_num}')

Done with 4


# Mirror circuits prediction errors

In [558]:
exp_num = 4
exp_path = f'./experiment_{exp_num}/'
pred_path = exp_path + '/predictions/'
inf_path = exp_path + '/processed_inputs_and_outputs/'
plot_path = '../../../paper-plots/simulations/'

In [559]:
exp_ylims = {0: [-6.5,10.5], 1: [-5.5, 15.5], 2: [-6, 8.5], 3: [-5, 10], 4: [-6, 10]}
exp_yticks = {0: [-6, -4, -2, 0, 2, 4, 6, 8, 10], 1: [-5, 0, 5, 10, 15], 2: [-4, 0, 4, 8],
              3: [-4, -2, 0, 2, 4, 6, 8], 4: [-4, -2, 0, 2, 4, 6, 8]}

exp_xlims = {0: [84, 100], 1: [84, 100], 2: [85, 100], 3: [84, 100], 4: [87, 100]}
exp_xticks = {0: np.linspace(85, 100, 4), 1: np.linspace(85, 100, 4), 2: np.linspace(85, 100, 4), 
              3: np.linspace(84, 100, 5), 4: np.linspace(88, 100, 4)}

In [560]:
dtypes = ['train', 'validate', 'test']

model_types = ['custom-nn', 'cnn']
circuits = 'test'

model_predictions = {'custom-nn': 'processed_custom_neural_net_mirrored_predictions.npz',
                     'cnn': 'processed_cnn_mirrored_predictions.npz'}

In [561]:
infidelities = np.load(inf_path + 'processed_mirrored_infidelities.npz')['infidelities']
spss = 100*(1 - infidelities / 10000)

m_predictions = {m: None for m in model_types}

for model in model_types:
    predictions = np.load(pred_path + model_predictions[model])['predictions']

    if model in ['custom-nn']:
        m_predictions[model] = 100*(1 - predictions / 10000) - spss
    else:
        m_predictions[model] = 100*(1 - predictions / 10000).reshape((-1,)) - spss

In [562]:
error_scatter_hist(ground_truth=spss, predictions=m_predictions,
             model_types = model_types, save = True, title = False, circuit_type='mirror-circuits')

In [563]:
print(f'Done with {exp_num}')

Done with 4
